In [ ]:
# # ----------------------------- Show All rows & columns -----------------------------

# import pandas as pd

# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)

In [ ]:
# # ----------------------------- play_sound -----------------------------

# import winsound
# import os

# import sys; sys.path.append('..')
# from config import MYPATHS

# def play_sound(file_name, error_message, raise_error):

#     # Alert_01, Alert_02, Atma_rama_Alarm
#     sound_path = os.path.join(MYPATHS['Audio_files'], f"{file_name}.wav")

#     if not os.path.isfile(sound_path):
#         print(f"\033[1;31mSound file not found: {sound_path}\033[0m")
#         if raise_error: raise FileNotFoundError(f"Missing sound file: {sound_path}")
#         return

#     winsound.PlaySound(sound_path, winsound.SND_FILENAME | winsound.SND_ASYNC)
#     print(f"\033[1;31m{error_message}\033[0m")

#     if raise_error:
#         raise Exception(f"\033[1;31m{error_message}\033[0m")

# play_sound(file_name = "Atma_rama_Alarm", error_message = "error_message", raise_error = False)


In [ ]:
# # ----------------------------- Get 1 minute data with retry -----------------------------

# import datetime
# import pandas as pd
# import time

# def get_data(security_id):
    
#     today = datetime.today().strftime("%Y-%m-%d")

#     for i in range(0, 3):
#         # Get the latest minute chart data
#         minute_chart = dhan.intraday_minute_data(
#             security_id=security_id,
#             exchange_segment="NSE_FNO", 
#             instrument_type="OPTIDX",
#             from_date=today,
#             to_date=today, 
#             interval=1
#         )

#         # Check if data is None or empty
#         if not minute_chart["data"]:
#             print("Your API subscription has expired or API related error", security_id)
#             continue

#         # Convert the data to a DataFrame
#         minute_chart_df = pd.DataFrame(minute_chart["data"])
#         minute_chart_df['datetime'] = pd.to_datetime(minute_chart_df['timestamp'], unit='s') + pd.Timedelta(hours=5, minutes=30)
#         minute_chart_df.drop(columns=['timestamp'], inplace=True)

#         if minute_chart_df is not None:
#             return minute_chart_df
        
#         time.sleep(0.14)
        

In [ ]:
# # ----------------------------- Get index of price filter effeciently from the dataframe -----------------------------

# def fast_select_three_rows(df, price_col):
#     # Convert to NumPy array for speed
#     prices = df[price_col].values
#     idx_below = (prices < 270).nonzero()[0]
    
#     if len(idx_below) == 0:
#         raise ValueError(f"No {price_col} < 270 found.")
    
#     # Get index of max price below 270
#     middle_pos = idx_below[prices[idx_below].argmax()]

#     # Collect surrounding indices
#     start = max(0, middle_pos - 1)
#     end = min(len(df), middle_pos + 2)

#     return df.iloc[start:end]

# # Usage:
# ce_selected = fast_select_three_rows(df, 'ce_last_price').copy()
# ce_selected.drop(columns=['pe_last_price', 'pe_iv', 'pe_oi', 'pe_volume', 'security_id_PE'], inplace=True)

# pe_selected = fast_select_three_rows(df, 'pe_last_price').copy()
# pe_selected.drop(columns=['security_id_CE', 'ce_iv', 'ce_volume', 'ce_oi', 'ce_last_price'], inplace=True)

In [ ]:
# # ----------------------------- Tick counts of folder's file -----------------------------

# # Calculate total tick count for all files in the specified folder using read_all_ticks
# import os
# import sys; sys.path.append('..'); from config import MYPATHS

# folder_path = MYPATHS['Simulation_Tick_Data'] + f"\\market_data_2025-06-26"
# total_tick_count = 0

# for file_name in os.listdir(folder_path):
#     if file_name.endswith(".bin"):
#         security_id = file_name.split(".")[0]
#         ticks = read_all_ticks(security_id, data_path=MYPATHS['base'], simulation_mode="Simulation")
#         total_tick_count += len(ticks)
#         print(f"Security ID: {security_id}, Tick Count: {len(ticks)}")

# print(f"Total tick count in folder: {total_tick_count}")

In [ ]:
# # ----------------------------- Check Internet Connection -----------------------------

# from datetime import datetime, timedelta
# import socket
# from IPython.display import display, HTML, Javascript
# import time

# def update_element(s):
#     display(HTML("""<div><pre id="clock" style='margin:0; padding:0;'></pre></div>"""))
#     display(Javascript(f"""document.getElementById('clock').innerHTML = '{s}';"""))

# def is_internet_available():

#     starting_time = str(datetime.now())
#     start_time = datetime.strptime(starting_time, "%Y-%m-%d %H:%M:%S.%f")

#     time_elapsed = None

#     while True:
#         try:
#             # Attempt to connect to Google's DNS server
#             socket.create_connection(("8.8.8.8", 53), timeout=3)
#             if time_elapsed is not None:
#                 print(f"Successfully connected to the internet after 0{time_elapsed}")
#             print(f"Internet Connection: 'Available'")
#             return True
#         except OSError:
            
#             current_time = datetime.now()
#             time_elapsed = str(current_time - start_time).split('.')[0]

#             update_element(f"Internet connection lost, retrying... Time Elapsed: 0{time_elapsed}")

#             time.sleep(1)

# is_internet_available()

In [ ]:
# # ----------------------------- Read Tick data from binary file -----------------------------

# from pathlib import Path
# import struct
# from datetime import datetime
# import sys; sys.path.append('..'); from config import MYPATHS

# class TickFileReader:
#     """Fast reader for tick files - supports parallel reading"""
    
#     RECORD_SIZE = 32

#     @staticmethod
#     def read_full_file(filepath):
#         """Read entire file - returns all ticks"""
#         ticks = []
#         try:
#             with open(filepath, 'rb') as f_file:
#                 while True:
#                     data = f_file.read(TickFileReader.RECORD_SIZE)
#                     if len(data) != TickFileReader.RECORD_SIZE:
#                         break
                    
#                     time_bytes, price, oi, volume = struct.unpack('8sddd', data)
#                     ltt = time_bytes.rstrip(b'\x00').decode('utf-8')
                    
#                     if ltt:
#                         ticks.append({
#                             'LTT': ltt,
#                             'price': price,
#                             'OI': oi,
#                             'volume': volume
#                         })
#         except Exception as e:
#             print(f"❌ Error reading full file: {e}")
        
#         return ticks

# def read_all_ticks(folder_path, file_name):
#     """Read all ticks for a security"""
#     return TickFileReader.read_full_file(Path(folder_path) / f"{file_name}.bin")

# date = "2025-06-26"
# folder_path = Path(MYPATHS['base']) / "Simulation_Tick_Data" / f"market_data_{date}"

# all_ticks = read_all_ticks(folder_path=folder_path, file_name="62371")
# print(f"Total ticks: {len(all_ticks)}")
# all_ticks

In [ ]:
# # ----------------------------- Convert .txt to .bin and .bin to .txt for raw simulation data-----------------------------

# import ast
# import pickle
# import struct
# from pathlib import Path
# import sys; sys.path.append('..'); from config import MYPATHS


# class SimulationDataConverter:
#     """Convert between simulation_data.txt and simulation_data.bin formats"""
    
#     def __init__(self):
#         pass
    
#     def convert_txt_to_bin(self, input_txt_path, output_bin_path=None):
#         """
#         Convert .txt simulation data to .bin format
        
#         Args:
#             input_txt_path: Path to input .txt file
#             output_bin_path: Path to output .bin file (optional, auto-generated if None)
#         """
#         input_path = Path(input_txt_path)
        
#         # Validate input file
#         if not input_path.exists():
#             raise FileNotFoundError(f"Input file not found: {input_path}")
        
#         if not input_path.suffix.lower() == '.txt':
#             raise ValueError(f"Input file must be .txt format, got: {input_path.suffix}")
        
#         # Generate output path if not provided
#         if output_bin_path is None:
#             output_bin_path = input_path.with_suffix('.bin')
#         else:
#             output_bin_path = Path(output_bin_path)
        
#         # Ensure output directory exists
#         output_bin_path.parent.mkdir(parents=True, exist_ok=True)
        
#         print(f"📄 Converting: {input_path.name}")
#         print(f"📁 Input:  {input_path}")
#         print(f"📁 Output: {output_bin_path}")
        
#         converted_count = 0
        
#         try:
#             with open(input_path, 'r') as txt_file, open(output_bin_path, 'wb') as bin_file:
#                 for line_num, line in enumerate(txt_file, 1):
#                     line = line.strip()
#                     if not line:
#                         continue
                    
#                     try:
#                         # Parse the dictionary string (same as simulator)
#                         tick_dict = ast.literal_eval(line)
                        
#                         # Convert to binary using pickle (same format as live data)
#                         binary_data = pickle.dumps(tick_dict)
                        
#                         # Write length prefix + data (same format as live data)
#                         bin_file.write(struct.pack('I', len(binary_data)))
#                         bin_file.write(binary_data)
                        
#                         converted_count += 1
                        
#                         # Progress indicator for large files
#                         if converted_count % 100000 == 0:
#                             print(f"📊 Converted {converted_count} records...")
                            
#                     except (ValueError, SyntaxError) as e:
#                         print(f"⚠️ Skipping malformed line {line_num}: {e}")
#                         continue
#                     except Exception as e:
#                         print(f"❌ Error processing line {line_num}: {e}")
#                         continue
            
#             print(f"✅ Conversion completed!")
#             print(f"📊 Total records converted: {converted_count}")
#             print(f"💾 Output file size: {output_bin_path.stat().st_size / (1024*1024):.2f} MB")
            
#             return {
#                 'input_file': str(input_path),
#                 'output_file': str(output_bin_path),
#                 'records_converted': converted_count,
#                 'output_size_mb': output_bin_path.stat().st_size / (1024*1024)
#             }
            
#         except Exception as e:
#             print(f"❌ Conversion failed: {e}")
#             # Clean up partial file on error
#             if output_bin_path.exists():
#                 output_bin_path.unlink()
#             raise
    
#     def convert_bin_to_txt(self, input_bin_path, output_txt_path=None):
#         """
#         Convert .bin simulation data to .txt format
        
#         Args:
#             input_bin_path: Path to input .bin file
#             output_txt_path: Path to output .txt file (optional, auto-generated if None)
#         """
#         input_path = Path(input_bin_path)
        
#         # Validate input file
#         if not input_path.exists():
#             raise FileNotFoundError(f"Input file not found: {input_path}")
        
#         if not input_path.suffix.lower() == '.bin':
#             raise ValueError(f"Input file must be .bin format, got: {input_path.suffix}")
        
#         # Generate output path if not provided
#         if output_txt_path is None:
#             output_txt_path = input_path.with_suffix('.txt')
#         else:
#             output_txt_path = Path(output_txt_path)
        
#         # Ensure output directory exists
#         output_txt_path.parent.mkdir(parents=True, exist_ok=True)
        
#         print(f"📄 Converting: {input_path.name}")
#         print(f"📁 Input:  {input_path}")
#         print(f"📁 Output: {output_txt_path}")
        
#         converted_count = 0
        
#         try:
#             with open(input_path, 'rb') as bin_file, open(output_txt_path, 'w') as txt_file:
#                 while True:
#                     try:
#                         # Read length prefix (4 bytes)
#                         length_data = bin_file.read(4)
#                         if len(length_data) != 4:
#                             break  # End of file
                        
#                         # Unpack length
#                         data_length = struct.unpack('I', length_data)[0]
                        
#                         # Read the pickled data
#                         pickled_data = bin_file.read(data_length)
#                         if len(pickled_data) != data_length:
#                             raise ValueError(f"Binary file corrupted: Expected {data_length} bytes, got {len(pickled_data)}")
                        
#                         # Unpickle the dictionary
#                         tick_dict = pickle.loads(pickled_data)
                        
#                         # Write as dictionary string (same format as original .txt)
#                         txt_file.write(str(tick_dict) + '\n')
                        
#                         converted_count += 1
                        
#                         # Progress indicator for large files
#                         if converted_count % 100000 == 0:
#                             print(f"📊 Converted {converted_count} records...")
                            
#                     except pickle.PickleError as e:
#                         print(f"❌ Pickle error at record {converted_count + 1}: {e}")
#                         break
#                     except struct.error as e:
#                         print(f"❌ Structure error at record {converted_count + 1}: {e}")
#                         break
#                     except Exception as e:
#                         print(f"❌ Error at record {converted_count + 1}: {e}")
#                         break
            
#             print(f"✅ Conversion completed!")
#             print(f"📊 Total records converted: {converted_count}")
#             print(f"💾 Output file size: {output_txt_path.stat().st_size / (1024*1024):.2f} MB")
            
#             return {
#                 'input_file': str(input_path),
#                 'output_file': str(output_txt_path),
#                 'records_converted': converted_count,
#                 'output_size_mb': output_txt_path.stat().st_size / (1024*1024)
#             }
            
#         except Exception as e:
#             print(f"❌ Conversion failed: {e}")
#             # Clean up partial file on error
#             if output_txt_path.exists():
#                 output_txt_path.unlink()
#             raise
    
#     def batch_convert_folder(self, folder_path, output_folder=None, convert_to='bin'):
#         """
#         Convert all files in a folder between .txt and .bin formats
        
#         Args:
#             folder_path: Path to folder containing files
#             output_folder: Output folder (optional, uses same folder if None)
#             convert_to: 'bin' to convert .txt to .bin, 'txt' to convert .bin to .txt
#         """
#         folder = Path(folder_path)
        
#         if not folder.exists() or not folder.is_dir():
#             raise ValueError(f"Invalid folder path: {folder}")
        
#         # Find files based on conversion direction
#         if convert_to == 'bin':
#             source_files = list(folder.glob("*.txt"))
#             source_ext = '.txt'
#             target_ext = '.bin'
#         elif convert_to == 'txt':
#             source_files = list(folder.glob("*.bin"))
#             source_ext = '.bin'
#             target_ext = '.txt'
#         else:
#             raise ValueError("convert_to must be 'bin' or 'txt'")
        
#         if not source_files:
#             print(f"📂 No {source_ext} files found in: {folder}")
#             return []
        
#         print(f"📂 Found {len(source_files)} {source_ext} files to convert to {target_ext}")
        
#         results = []
        
#         for source_file in sorted(source_files):
#             try:
#                 if output_folder:
#                     output_path = Path(output_folder) / source_file.with_suffix(target_ext).name
#                 else:
#                     output_path = source_file.with_suffix(target_ext)
                
#                 if convert_to == 'bin':
#                     result = self.convert_txt_to_bin(source_file, output_path)
#                 else:
#                     result = self.convert_bin_to_txt(source_file, output_path)
                
#                 results.append(result)
                
#             except Exception as e:
#                 print(f"❌ Failed to convert {source_file.name}: {e}")
#                 continue
        
#         print(f"🎯 Batch conversion completed: {len(results)}/{len(source_files)} files converted")
#         return results
    
#     def verify_conversion(self, original_path, converted_path):
#         """
#         Verify that conversion was successful by comparing record counts
        
#         Args:
#             original_path: Path to original file (.txt or .bin)
#             converted_path: Path to converted file (.bin or .txt)
#         """
#         print(f"🔍 Verifying conversion...")
        
#         original_path = Path(original_path)
#         converted_path = Path(converted_path)
        
#         original_count = 0
#         converted_count = 0
        
#         # Count records in original file
#         if original_path.suffix.lower() == '.txt':
#             with open(original_path, 'r') as f:
#                 for line in f:
#                     if line.strip():
#                         original_count += 1
        
#         elif original_path.suffix.lower() == '.bin':
#             with open(original_path, 'rb') as f:
#                 while True:
#                     try:
#                         length_data = f.read(4)
#                         if len(length_data) != 4:
#                             break
#                         data_length = struct.unpack('I', length_data)[0]
#                         pickled_data = f.read(data_length)
#                         if len(pickled_data) != data_length:
#                             break
#                         pickle.loads(pickled_data)
#                         original_count += 1
#                     except:
#                         break
        
#         # Count records in converted file
#         if converted_path.suffix.lower() == '.txt':
#             with open(converted_path, 'r') as f:
#                 for line in f:
#                     if line.strip():
#                         converted_count += 1
        
#         elif converted_path.suffix.lower() == '.bin':
#             with open(converted_path, 'rb') as f:
#                 while True:
#                     try:
#                         length_data = f.read(4)
#                         if len(length_data) != 4:
#                             break
#                         data_length = struct.unpack('I', length_data)[0]
#                         pickled_data = f.read(data_length)
#                         if len(pickled_data) != data_length:
#                             break
#                         pickle.loads(pickled_data)
#                         converted_count += 1
#                     except:
#                         break
        
#         if original_count == converted_count:
#             print(f"✅ Verification successful: {converted_count} records match")
#             return True
#         else:
#             print(f"❌ Verification failed: Original has {original_count} records, Converted has {converted_count} records")
#             return False


# # Convenience functions for easy usage
# def convert_txt_to_bin(txt_file_path, bin_file_path=None):
#     """Convert single .txt file to .bin format"""
#     converter = SimulationDataConverter()
#     return converter.convert_txt_to_bin(txt_file_path, bin_file_path)

# def convert_bin_to_txt(bin_file_path, txt_file_path=None):
#     """Convert single .bin file to .txt format"""
#     converter = SimulationDataConverter()
#     return converter.convert_bin_to_txt(bin_file_path, txt_file_path)

# def convert_folder_to_bin(folder_path, output_folder=None):
#     """Convert all .txt files in folder to .bin format"""
#     converter = SimulationDataConverter()
#     return converter.batch_convert_folder(folder_path, output_folder, 'bin')

# def convert_folder_to_txt(folder_path, output_folder=None):
#     """Convert all .bin files in folder to .txt format"""
#     converter = SimulationDataConverter()
#     return converter.batch_convert_folder(folder_path, output_folder, 'txt')

# def verify_conversion(original_file, converted_file):
#     """Verify conversion was successful"""
#     converter = SimulationDataConverter()
#     return converter.verify_conversion(original_file, converted_file)

# # Example usage
# if __name__ == "__main__":

#     input_file_path = MYPATHS['Raw_data_bin'] + f"\\2025-06-26.bin"
#     output_file_path = MYPATHS['Raw_data_txt'] + f"\\2025-06-26.txt"

#     # # Convert .txt to .bin
#     # convert_txt_to_bin(input_file_path, output_file_path)
    
#     # Convert .bin to .txt
#     # convert_bin_to_txt(input_file_path, output_file_path)
    
#     # # Convert folder of .txt files to .bin
#     # convert_folder_to_bin(r"D:\example\txt_files")
    
#     # # Convert folder of .bin files to .txt
#     # convert_folder_to_txt(r"D:\example\bin_files")
    
#     # Verify conversion
#     verify_conversion(input_file_path, output_file_path)
    
#     pass

In [ ]:
# # ----------------------------- Read Raw simulation data from binary file -----------------------------

# import pickle
# import struct
# from pathlib import Path
# import sys; sys.path.append('..'); from config import MYPATHS


# class BinFileReader:
#     """Ultra-fast binary file reader for simulation data"""
    
#     @staticmethod
#     def read_bin_file(file_path):
#         """
#         Read complete .bin file and return all tick data
        
#         Args:
#             file_path: Path to .bin file
            
#         Returns:
#             list: List of all tick dictionaries
#         """
#         file_path = Path(file_path)
        
#         if not file_path.exists():
#             raise FileNotFoundError(f"File not found: {file_path}")
        
#         if not file_path.suffix.lower() == '.bin':
#             raise ValueError(f"File must be .bin format, got: {file_path.suffix}")
        
#         ticks = []
        
#         with open(file_path, 'rb') as f:
#             while True:
#                 # Read length prefix (4 bytes)
#                 length_data = f.read(4)
#                 if len(length_data) != 4:
#                     break  # End of file
                
#                 # Unpack length
#                 data_length = struct.unpack('I', length_data)[0]
                
#                 # Read pickled data
#                 pickled_data = f.read(data_length)
#                 if len(pickled_data) != data_length:
#                     raise ValueError(f"Corrupted file: Expected {data_length} bytes, got {len(pickled_data)}")
                
#                 # Unpickle and add to list
#                 tick_dict = pickle.loads(pickled_data)
#                 ticks.append(tick_dict)
        
#         return ticks
    
#     @staticmethod
#     def read_bin_file_generator(file_path):
#         """
#         Memory-efficient generator to read .bin file one tick at a time
        
#         Args:
#             file_path: Path to .bin file
            
#         Yields:
#             dict: Individual tick dictionary
#         """
#         file_path = Path(file_path)
        
#         if not file_path.exists():
#             raise FileNotFoundError(f"File not found: {file_path}")
        
#         with open(file_path, 'rb') as f:
#             while True:
#                 # Read length prefix
#                 length_data = f.read(4)
#                 if len(length_data) != 4:
#                     break
                
#                 # Read and unpickle data
#                 data_length = struct.unpack('I', length_data)[0]
#                 pickled_data = f.read(data_length)
                
#                 if len(pickled_data) != data_length:
#                     raise ValueError(f"Corrupted file at position {f.tell()}")
                
#                 yield pickle.loads(pickled_data)

# # Convenience functions
# def read_full_bin(file_path):
#     """Read complete .bin file - returns all ticks"""
#     return BinFileReader.read_bin_file(file_path)

# def read_bin_streaming(file_path):
#     """Memory-efficient streaming read - yields one tick at a time"""
#     return BinFileReader.read_bin_file_generator(file_path)

# # -------------------------------------

# # Example usage
# file_path = MYPATHS['Raw_data_bin'] + f"\\2025-06-30.bin"

# # # # Method 1: Read all data at once (fast, uses more memory)
# # all_ticks = read_full_bin(file_path)
# # print(f"Loaded {len(all_ticks)} ticks")

# # # Method 2: Stream data one by one (memory efficient)
# # for i, tick in enumerate(read_bin_streaming(file_path)):
# #     print(f"Tick {i}: {tick}")
# #     # print(tick)
# #     if i >= 10:
# #         break


In [ ]:
# # ----------------------------- Preventing the Internet Connection Loss -----------------------------

# import keyboard
# import random
# import time

# def prevent_sleep_with_win_search(interval=150):

#     # Sample search terms (you can expand this list)
#     search_terms = [
#         "cmd", "notepad", "calc", "python", "edge",
#         "paint", "settings", "task manager", "snipping tool",
#         "control panel", "windows update", "event viewer"
#     ]

#     try:
#         while True:
#             term = random.choice(search_terms)

#             # Press Windows key
#             keyboard.press_and_release('windows')
#             time.sleep(0.5)

#             # Type search term
#             keyboard.write(term, delay=0.05)
#             time.sleep(1)

#             # Press Esc to close Start menu
#             keyboard.press_and_release('esc')

#             # Wait until next iteration
#             time.sleep(interval)

#     except KeyboardInterrupt:
#         print("\nStopped.")

# # Run it
# prevent_sleep_with_win_search(interval=5)
